# Numerical accuracy of uvcontsub2021

This notebook simulates MeasurementSets with known continuum shapes (for example, as polynomials of different order with known coefficients). It has functions to produce MSs including point sources, spectral lines, added noise, and polynomial continuum functions. These MSs can be used to test the numerical accuracy of the task uvcontsub2021, illustrate its usage and experiment with it under different simulated conditions. 

## Import required CASA tasks and tools, and other packages

In [ ]:
from casatools import componentlist, ctsys, measures, simulator, table
from casatasks import flagdata, listobs
from casatasks import uvcontsub2021

from casatasks.private import simutil

from matplotlib import colors as mcolors

import pprint
import numpy as np
import glob
import os, shutil
import scipy
import matplotlib.pyplot as plt
import numpy as np

cl = componentlist()
sm = simulator()

# Definition of simulation building blocks
This section defines function to:
* Create a MeasurementSet with basic structure
* Plot visibilities
* Populate and edit the visibilities of an MS (add sources, continuum, noise, etc.)

Uses functions from https://github.com/urvashirau/Simulation-in-CASA/blob/master/Basic_Simulation_Demo/Simulation_Script_Demo.ipynb, adapted to this set of simulations for uvcontsub.

In [ ]:
def make_ms_frame(msname:str, ant_config:str, spw_params=None, verbose=False):
    """ 
    Construct an empty MeasurementSet that has the desired observation setup. 
        
    Args:
        msname: MS to create
        ant_config: a telescope configuration from casadata (alma/simmos) (for 
                  example  alma/simmos/aca.cycle8.cfg)
        swp_params: parameters such as SPW name, number of channels, frequency
    """ 
            
    ## Open the simulatorFunctions
    sm.open(ms=msname);

    ## Read/create an antenna configuration. 
    ## Canned antenna config text files are located here: <casadata>/alma/simmos/*cfg
        
    antennalist = os.path.join(ctsys.resolve("alma/simmos"), ant_config)
    if verbose:
        print(f'Using antenna list file: {antennalist}')
            
        
    ## Fictitious telescopes can be simulated by specifying x, y, z, d, an, telname, antpos.
    ##     x,y,z are locations in meters in ITRF (Earth centered) coordinates. 
    ##     d, an are lists of antenna diameter and name.
    ##     telname and obspos are the name and coordinates of the observatory. 
        
    mysu = simutil.simutil()
    (x, y, z, d, an, an2, telname, obspos) = mysu.readantenna(antennalist)

    ## Set the antenna configuration
    metool = measures()
    sm.setconfig(
        telescopename=telname,
        x=x,
        y=y,
        z=z,
        dishdiameter=d,
        mount=['alt-az'], 
        antname=an,
        coordsystem='local',
        referencelocation=metool.observatory(telname)
    )

    ## Set the polarization mode (this goes to the FEED subtable)
    sm.setfeed(mode='perfect X Y', pol=['']);    # X Y / R L

    ## Set the spectral window and polarization (one data-description-id). 
    ## Call multiple times with different names for multiple SPWs or pol setups.
    sm.setspwindow(
        spwname=spw_params['name'],
        freq=spw_params['freq'],
        deltafreq='0.1GHz',
        freqresolution='0.2GHz',
        nchannels=spw_params['nchan'],
        stokes='XX YY'
    )

    ## Setup source/field information (i.e. where the observation phase center is)
    ## Call multiple times for different pointings or source locations.
    source_name = 'simulated_source'
    sm.setfield(
        sourcename=source_name,
        sourcedirection=metool.direction(
            rf='J2000',
            v0='19h59m28.5s',
            v1='+40d44m01.5s'
        )
    )

    ## Set shadow/elevation limits (if you care). These set flags.
    sm.setlimits(shadowlimit=0.01, elevationlimit='1deg');

    ## Leave autocorrelations out of the MS.
    sm.setauto(autocorrwt=0.0);  

    ## Set the integration time, and the convention to use for timerange specification
    ## Note : It is convenient to pick the hourangle mode as all times specified in sm.observe()
    ##        will be relative to when the source transits.
    sm.settimes(
        integrationtime='2s',
        usehourangle=True,
        referencetime=metool.epoch('UTC','2021/10/14/00:01:02')
    )

    ## Construct MS metadata and UVW values for one scan and did 
    ## Call multiple times for multiple scans.
    ## Call this with different sourcenames (fields) and spw/pol settings as defined above.
    ## Timesteps will be defined in intervals of 'integrationtime', between starttime and stoptime.
        
    sm.observe(
        sourcename=source_name,
        spwname=spw_params['name'],
        starttime='0', 
        stoptime='+8s'
    )

    ## Close the simulator
    sm.close()
      
    ## Unflag everything (unless you care about elevation/shadow flags)
    flagdata(vis=msname, mode='unflag', flagbackup=False)
    
    
def add_gaussian_noise(msname:str, noise_level_sigma='0.1Jy'):
    """
    Adds Gaussian random noise using simulator / corrupt
  
    Args:
        ms_name: MS to modify
        noise_level_sigma: noise sigma as used in the simulator corrupt function
    """
        
    try:
        sm.openfromms(msname)
        sm.setseed(4321)
        sm.setnoise(mode='simplenoise', simplenoise=noise_level_sigma)
        sm.corrupt()
    finally:
        sm.close()
            
    
def make_point_source_comp_list(cl_name:str, freq:str, flux:str, spectrumtype:str, index:int, 
                                direction='J2000 19h59m28.5s +40d44m01.5s',
                                fluxunit='Jy', shape='point', label='sim_point_source'):
    """
    Makes a component list file with a point source
  
    Args:
        cl_name: name of the component list file to create
        freq: freq quantity (with units) as used in the component list tool 
        flux: flux, units are assumed in Jy
        spectrumtype: as used in component list tool: constant/spectral index
        index: spectral index
    """
        
    try:
        cl.addcomponent(
            dir=direction, 
            flux=flux,
            fluxunit=fluxunit, 
            freq=freq,
            shape=shape,     # shape='gaussian',
                             # majoraxis="5.0arcmin", minoraxis='2.0arcmin',                                
                             # polarization='linear', / 'Stokes'
                             # spectrumtype:'spectral index' / 'constant'
            spectrumtype=spectrumtype,
            index=index,
            label=label)
        cl.rename(filename=cl_name)
    finally:
        cl.close()
        
def sim_from_comp_list(msname:str, cl_name:str):
    """
    Updates the MS visibilities using simulator.predict to add
    components from the components list

    Args:
        ms_name: MS to modify
        cl_name: name of components list file to simulate
    """
    try:
        sm.openfromms(msname)
        sm.predict(complist=cl_name, incremental=False)
    finally:
        sm.close()
            
def add_point_source_component(msname, cl_name = 'sim_point_source.cl', freq=None, 
                                flux=5.0, spectrumtype='constant', index=-1):
    """
    Adds a point source to the MS
 
    Args:
        ms_name: MS to modify
        freq: 
        spectrumtype:
        flux: In Jy, as used in componentlist.addcomponent
    """
        
    make_point_source_comp_list(
        cl_name=cl_name, 
        freq=freq, 
        flux=flux,  
        spectrumtype=spectrumtype, 
        index=index
    )
        
    sim_from_comp_list(msname, cl_name)
    shutil.rmtree(cl_name)
        
def add_spectral_line(msname:str, line:'numpy.array', chan_range=[60, 86], amp_factor=None):
    """
    Adds a spectral line as a Gaussian function in the range of channels given
  
    Args:
        ms_name: MS to modify
        line: function to produce spectral line. ex: fn(x, mu, sigma)
        chan_range: list with indices of the first and last channel
        amp_factor: factor to multiply the peak height / flux density
    """
        
    try:
        tbtool = table()
        tbtool.open(msname, nomodify=False)
        data = tbtool.getcol('DATA')

            
            
        if not amp_factor:
            amp_factor = 1.0/np.max(line)
            
        data[:,chan_range[0]:chan_range[1],:] += amp_factor * (1+0j) * line.reshape((1, len(line), 1))
        tbtool.putcol('DATA', data)
        
    finally:
        tbtool.done()
            
def add_polynomial_continuum(msname:str, pol_coeffs:list, nchan:int, amp_factor=1.0):
    """
    Update MS visibilities adding a polynomial evaluated for all channels.
        
    Args:
        ms_name: MS to modify
        pol_coeff: polynomial coefficients, evaluated [0.5, 0.25] => 0.5x + 0.25
        nchan: number of channels in the SPW (x axis to eval polynomial)
    """
    try:
        tbtool = table()
        tbtool.open(msname, nomodify=False)
        data = tbtool.getcol('DATA')

        x = np.linspace(0, 1, nchan)
        polynomial = np.polyval(pol_coeffs, x)
            
        # Add same polynomial to real and imag part
        data += amp_factor * (1+1j) * polynomial.reshape((1, len(polynomial),1))

        tbtool.putcol('DATA', data)
    finally:
        tbtool.done()
        
def plot_ms_data(msname='sim_data.ms', myplot='uv', fitline=None, average=False)->None:
    """
    Options : myplot='uv'
              myplot='data_spectrum'
    Args:
      plot_complex: 'abs', 'real', or 'imag'
    """
    from casatools import table
    from matplotlib import colors as mcolors
    
    import matplotlib.pyplot as plt
    import numpy as np

    tb = table()
    from matplotlib.collections import LineCollection
    tb.open(msname)

    # UV coverage plot
    if myplot=='uv':
        plt.figure(figsize=(4,4))
        plt.clf()
        
        uvw = tb.getcol('UVW')
        
        plt.plot( uvw[0], uvw[1], '.')
        plt.plot( -uvw[0], -uvw[1], '.')
        plt.title('UV Coverage')
    
    # Spectrum of chosen column. Make a linecollection out of each row in the MS.
    spectrum = ['data_spectrum', 'corr_spectrum', 'resdata_spectrum', 'model_spectrum']
    
    if myplot in spectrum:
        dats=None
        if myplot=='data_spectrum':
            dats = tb.getcol('DATA')
        if myplot=='corr_spectrum':
            dats = tb.getcol('CORRECTED_DATA')
        if myplot=='resdata_spectrum':
            dats = tb.getcol('DATA') - tb.getcol('MODEL_DATA') 
        if myplot=='rescorr_spectrum':
            dats = tb.getcol('CORRECTED_DATA') - tb.getcol('MODEL_DATA') 
        if myplot=='model_spectrum':
            dats = tb.getcol('MODEL_DATA')
            
        xs = np.zeros((dats.shape[2],dats.shape[1]),'int')
        for chan in range(0,dats.shape[1]):
            xs[:,chan] = chan
    
        npl = dats.shape[0]
        fig, ax = plt.subplots(2, npl, figsize=(30, 20))

        if average:
            for pol in range(0,dats.shape[0]):
                x = xs
                y_real = np.mean(dats[pol,:,:].real.T, axis=0)
                y_imag = np.mean(dats[pol,:,:].imag.T, axis=0)
                
                y_real_std = np.std(dats[pol,:,:].real.T, axis=0)
                y_imag_std = np.std(dats[pol,:,:].imag.T, axis=0)
            
                ax[pol, 0].scatter(x=xs[0], y=y_real, c='#ff8d33')
                ax[pol, 0].plot(xs[0], y_real, c='#ff8d33')
                
                if fitline is not None:
                    y_fit = np.mean(fitline[pol,:,:].real.T, axis=0)
                    ax[pol, 0].plot(xs[0], y_fit, color='black', linewidth=3)
                
                ax[pol, 0].fill_between(
                    x=xs[0], 
                    y1=y_real+y_real_std,
                    y2=y_real-y_real_std,
                    alpha=0.2,
                    edgecolor='#ff8d33',
                    facecolor='#f0b27a'
                )
                ax[pol, 0].set_title(myplot + ': polar: '+ str(pol) + " (real)")
                ax[pol, 0].set_xlim(x.min(), x.max())
                ax[pol, 0].set_ylim(dats[pol,:,:].real.T.min(), dats[pol,:,:].real.T.max())
            
                ax[pol, 1].scatter(x=xs[0], y=y_imag, c='#3933ff')
                ax[pol, 1].plot(xs[0], y_imag, c='#3933ff')
                
                if fitline is not None:
                    y_fit = np.mean(fitline[pol,:,:].imag.T, axis=0)
                    ax[pol, 1].plot(xs[0], y_fit, color='black', linewidth=3)
                
                ax[pol, 1].fill_between(
                    x=xs[0], 
                    y1=y_imag + y_imag_std,
                    y2=y_imag - y_imag_std,
                    alpha=0.2,
                    edgecolor='#3933ff',
                    facecolor='#85c1e9'
                )
                ax[pol, 1].set_title(myplot + ': polar: '+ str(pol) + ": (imaginary)")
                ax[pol, 1].set_xlim(x.min(), x.max())
                ax[pol, 1].set_ylim(dats[pol,:,:].imag.T.min(), dats[pol,:,:].imag.T.max())
            
        else:
            colors = [
                mcolors.to_rgba(c) for c in plt.rcParams['axes.prop_cycle'].by_key()['color']
            ]
        
            for pol in range(0,dats.shape[0]):
                x = xs
                y_real = dats[pol,:,:].real.T 
                y_imag = dats[pol,:,:].imag.T 
            
                data_real = np.stack((x, y_real), axis=2)
                data_imag = np.stack((x, y_imag), axis=2)
            
                ax[pol, 0].add_collection(LineCollection(data_real, colors=colors))
                ax[pol, 0].set_title(myplot + ': polar: '+ str(pol) + " (real)")
                ax[pol, 0].set_xlim(x.min(), x.max())
                ax[pol, 0].set_ylim(y_real.min(), y_real.max())
            
                ax[pol, 1].add_collection(LineCollection(data_imag, colors=colors))
                ax[pol, 1].set_title(myplot + ': polar: '+ str(pol) + ": (imaginary)")
                ax[pol, 1].set_xlim(x.min(), x.max())
                ax[pol, 1].set_ylim(y_imag.min(), y_imag.max())
        
        plt.show()
        
def get_spectrum_data(msname:str, myplot:str)->'numpy.array':
    """
              myplot='data_spectrum'
      Args:
          plot_complex: 'abs', 'real', or 'imag'
    """
    from casatools import table
    import numpy as np

    tb = table()
    from matplotlib.collections import LineCollection
    tb.open(msname)
    
    data=None
    
    spectrum = ['data_spectrum', 'corr_spectrum', 'resdata_spectrum', 'model_spectrum']
    # Spectrum of chosen column. Make a linecollection out of each row in the MS.
    if myplot in spectrum:
        if myplot=='data_spectrum':
            data = tb.getcol('DATA')
        if myplot=='corr_spectrum':
            data = tb.getcol('CORRECTED_DATA')
        if myplot=='resdata_spectrum':
            data = tb.getcol('DATA') - tb.getcol('MODEL_DATA') 
        if myplot=='rescorr_spectrum':
            data = tb.getcol('CORRECTED_DATA') - tb.getcol('MODEL_DATA') 
        if myplot=='model_spectrum':
            data = tb.getcol('MODEL_DATA')
  
    return data

def get_continuum_data(msname:str, myplot:str, chan_list=list)->'numpy.array':
    
    baseline = None
    mask = []
    
    # Retrieve full data column from measurement file
    data = get_spectrum_data(msname, myplot)
    
    # Parse data column and append only background
    for lower, upper in chan_list:
        for chan in range(lower-1, upper):
            mask.append(chan)
    
    # Apply channel mask to get baseline
    baseline = np.delete(data, mask, axis=1)
            
    return baseline

def plot_baseline_histogram(msname:str, myplot:str, chan_list:list):    
    baseline = get_continuum_data(
        msname=msname, 
        myplot='data_spectrum', 
        chan_list=[chan_list]
    )
    
    npl = baseline.shape[0]
    
    fig, ax = plt.subplots(2, npl, figsize=(30, 20))
    
    bins = 100
    
    for polar in range(npl):
        collapsed = baseline[polar, :, :].flatten()
        
        # Histogram of corrected background: real
        _, hist_bins, _ = ax[polar, 0].hist(collapsed.real, bins, color='#f0b27a', density=True)
        ax[polar, 0].set_title("Corrected baseline (real): polarization: {}".format(polar))
        
        mu, sigma = scipy.stats.norm.fit(collapsed.real)
        fit_line = scipy.stats.norm.pdf(hist_bins, mu, sigma)
        
        textstr = '\n'.join((
            r'$\mu=%.3f$' % (mu, ),
            r'$\sigma=%.3f$' % (sigma, )))
        
        props = dict(boxstyle='round', facecolor='#f0b27a', alpha=0.5)
        ax[polar, 0].text(0.05, 0.95, textstr, 
                          transform=ax[polar, 0].transAxes, fontsize=16,
                          verticalalignment='top', bbox=props)
        
        ax[polar, 0].plot(hist_bins, fit_line, color='black', linewidth=2)
        
        # Histogram of corrected background: imaginary
        _, hist_bins, _ = ax[polar, 1].hist(collapsed.imag, bins, color='#85c1e9', density=True)
        ax[polar, 1].set_title("Corrected baseline (imaginary): polarization: {}".format(polar))
        
        mu, sigma = scipy.stats.norm.fit(collapsed.imag)
        fit_line = scipy.stats.norm.pdf(hist_bins, mu, sigma)
        
        textstr = '\n'.join((
            r'$\mu=%.3f$' % (mu, ),
            r'$\sigma=%.3f$' % (sigma, )))
        
        props = dict(boxstyle='round', facecolor='#85c1e9', alpha=0.5)
        ax[polar, 1].text(0.05, 0.95, textstr, 
                          transform=ax[polar, 1].transAxes, fontsize=16,
                          verticalalignment='top', bbox=props)
        
        ax[polar, 1].plot(hist_bins, fit_line, color='black', linewidth=2)
        
        
    plt.show()

## Simple MS with continuum, line, and noise, plotted step by step

Using order 0 continuum.

The structure of this MS is the one used at the moment in the task test (test_task_req_uvcontsub2021). nchan=128. The spectral line is added to channels 60-85.
The value of the fitspw parameter for this MS structure would be '0:0\~59;86\~127'

### Polynomial n=0 continuum

In [ ]:
ant_config = 'alma.cycle8.8.cfg'
spw_params = {
    'name': "Simulated_Band",
    'freq': '150GHz',
    'nchan': 128
}

msname='sim_alma_noise_cont_poly_order_0.ms' 

make_ms_frame(
    msname=msname, 
    ant_config=ant_config, 
    spw_params=spw_params, 
    verbose=True
)

# Add point source to measurement set
add_point_source_component(
    msname=msname,
    cl_name = 'sim_point_source.cl', 
    freq='100GHz', 
    flux=0.5, 
    spectrumtype='constant', 
    index=-1
)

# Define the spectral line shape
gaussian = lambda x, mu, sigma: np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))
x = np.linspace(-3, 3, 86-60)

# Add spectral line
add_spectral_line(
    msname=msname,
    line=gaussian(x, 0., 0.58), 
    chan_range=[60, 86]
)

# Add polynomial component to background
add_polynomial_continuum(
    msname=msname,
    pol_coeffs=[0.025], 
    nchan = spw_params['nchan'], 
    amp_factor=1.0)

# Add gaussian noise to all channels
add_gaussian_noise(
    msname=msname, 
    noise_level_sigma='0.1Jy')

# Make diagnostic plots
plot_ms_data(
    msname=msname,
    myplot='data_spectrum'
)

### Polynomial n=1 continuum

In [ ]:
ant_config = 'alma.cycle8.8.cfg'
spw_params = {
    'name': "Simulated_Band",
    'freq': '150GHz',
    'nchan': 128
}

msname='sim_alma_noise_cont_poly_order_1.ms' 

make_ms_frame(
    msname=msname, 
    ant_config=ant_config, 
    spw_params=spw_params, 
    verbose=True
)

# Add point source to measurement set
add_point_source_component(
    msname=msname,
    cl_name = 'sim_point_source.cl', 
    freq='100GHz', 
    flux=0.5, 
    spectrumtype='constant', 
    index=-1
)

# Define the spectral line shape
gaussian = lambda x, mu, sigma: np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))
x = np.linspace(-3, 3, 86-60)

# Add spectral line
add_spectral_line(
    msname=msname,
    line=gaussian(x, 0., 0.58), 
    chan_range=[60, 86]
)

# Add polynomial component to background
add_polynomial_continuum(
    msname=msname,
    pol_coeffs=[-1, 0.75], 
    nchan = spw_params['nchan'], 
    amp_factor=1.0)

# Add gaussian noise to all channels
add_gaussian_noise(
    msname=msname, 
    noise_level_sigma='0.1Jy')

# Make diagnostic plots
plot_ms_data(
    msname=msname,
    myplot='data_spectrum'
)

### Polynomial n=2 continuum

In [ ]:
ant_config = 'alma.cycle8.8.cfg'
spw_params = {
    'name': "Simulated_Band",
    'freq': '150GHz',
    'nchan': 128
}

msname='sim_alma_noise_cont_poly_order_2.ms' 

make_ms_frame(
    msname=msname, 
    ant_config=ant_config, 
    spw_params=spw_params, 
    verbose=True
)

# Add point source to measurement set
add_point_source_component(
    msname=msname,
    cl_name = 'sim_point_source.cl', 
    freq='100GHz', 
    flux=0.5, 
    spectrumtype='constant', 
    index=-1
)

# Define the spectral line shape
gaussian = lambda x, mu, sigma: np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))
x = np.linspace(-3, 3, 86-60)

# Add spectral line
add_spectral_line(
    msname=msname,
    line=gaussian(x, 0., 0.58), 
    chan_range=[60, 86]
)

# Add polynomial component to background
add_polynomial_continuum(
    msname=msname,
    pol_coeffs=[-1., 1, 0.15], 
    nchan = spw_params['nchan'], 
    amp_factor=1.0)

# Add gaussian noise to all channels
add_gaussian_noise(
    msname=msname, 
    noise_level_sigma='0.1Jy')

# Make diagnostic plots
plot_ms_data(
    msname=msname,
    myplot='data_spectrum'
)

### Polynomial n=3 continuum

In [ ]:
ant_config = 'alma.cycle8.8.cfg'
spw_params = {
    'name': "Simulated_Band",
    'freq': '150GHz',
    'nchan': 128
}

msname='sim_alma_noise_cont_poly_order_3.ms' 

make_ms_frame(
    msname=msname, 
    ant_config=ant_config, 
    spw_params=spw_params, 
    verbose=True
)

# Add point source to measurement set
add_point_source_component(
    msname=msname,
    cl_name = 'sim_point_source.cl', 
    freq='100GHz', 
    flux=0.5, 
    spectrumtype='constant', 
    index=-1
)

# Define the spectral line shape
gaussian = lambda x, mu, sigma: np.exp(-np.power(x - mu, 2.) / (2 * np.power(sigma, 2.)))
x = np.linspace(-3, 3, 86-60)

# Add spectral line
add_spectral_line(
    msname=msname,
    line=gaussian(x, 0., 0.58), 
    chan_range=[60, 86]
)

# Add polynomial component to background
add_polynomial_continuum(
    msname=msname,
    pol_coeffs=[1., -0.75, -0.25, 0.15], 
    nchan = spw_params['nchan'], 
    amp_factor=1.0)

# Add gaussian noise to all channels
add_gaussian_noise(
    msname=msname, 
    noise_level_sigma='0.1Jy')

# Make diagnostic plots
plot_ms_data(
    msname=msname,
    myplot='data_spectrum'
)

## uvcontsub

In [ ]:
from casatasks import uvcontsub2021
fitspw='0:0~59;86~127'
  
ms_uvcont = 'uvcont_noise_sub_0.ms'
res = uvcontsub2021(
    vis='sim_alma_noise_cont_poly_order_0.ms', 
    outputvis=ms_uvcont, 
    fitorder=0, 
    fitspw=fitspw, 
    writemodel=True
)

ms_uvcont = 'uvcont_noise_sub_1.ms'
res = uvcontsub2021(
    vis='sim_alma_noise_cont_poly_order_1.ms', 
    outputvis=ms_uvcont, 
    fitorder=1, 
    fitspw=fitspw, 
    writemodel=True
)

ms_uvcont = 'uvcont_noise_sub_2.ms'
res = uvcontsub2021(
    vis='sim_alma_noise_cont_poly_order_2.ms', 
    outputvis=ms_uvcont, 
    fitorder=2, 
    fitspw=fitspw, 
    writemodel=True
)

ms_uvcont = 'uvcont_noise_sub_3.ms'
res = uvcontsub2021(
    vis='sim_alma_noise_cont_poly_order_3.ms', 
    outputvis=ms_uvcont, 
    fitorder=3, 
    fitspw=fitspw, 
    writemodel=True
)

## Average Plots

### polynomial n=0 continuum

In [ ]:
fit_poly_0 = get_spectrum_data(
    msname='uvcont_noise_sub_0.ms', 
    myplot='model_spectrum'
)

plot_ms_data(
    msname='sim_alma_noise_cont_poly_order_0.ms',
    myplot='data_spectrum',
    fitline=fit_poly_0,
    average=True
)

### Corrected polynomial n=0 continuum

In [ ]:
plot_ms_data(
    msname='uvcont_noise_sub_0.ms',
    myplot='data_spectrum',
    average=True
)

### polynomial n=1 continuum

In [ ]:
fit_poly_1 = get_spectrum_data(
    msname='uvcont_noise_sub_1.ms', 
    myplot='model_spectrum'
)

plot_ms_data(
    msname='sim_alma_noise_cont_poly_order_1.ms',
    myplot='data_spectrum',
    fitline=fit_poly_1,
    average=True
)

### Corrected polynomial n=1continuum

In [ ]:
plot_ms_data(
    msname='uvcont_noise_sub_1.ms',
    myplot='data_spectrum',
    average=True
)

### polynomial n=2 continuum

In [ ]:
fit_poly_2 = get_spectrum_data(
    msname='uvcont_noise_sub_2.ms', 
    myplot='model_spectrum'
)

plot_ms_data(
    msname='sim_alma_noise_cont_poly_order_2.ms',
    myplot='data_spectrum',
    fitline=fit_poly_2,
    average=True
)

### Corrected polynomial n=2 continuum

In [ ]:
plot_ms_data(
    msname='uvcont_noise_sub_2.ms',
    myplot='data_spectrum',
    fitline=fit_poly_2,
    average=True
)

### polynomial n=3 continuum

In [ ]:
fit_poly_3 = get_spectrum_data(
    msname='uvcont_noise_sub_3.ms', 
    myplot='model_spectrum'
)

plot_ms_data(
    msname='sim_alma_noise_cont_poly_order_3.ms',
    myplot='data_spectrum',
    fitline=fit_poly_3,
    average=True
)

### Corrected polynomial n=3 continuum

In [ ]:
plot_ms_data(
    msname='uvcont_noise_sub_3.ms',
    myplot='data_spectrum',
    average=True
)

## Baseline 

### polynomial n=0 continuum

In [ ]:
msname='uvcont_noise_sub_0.ms'

plot_baseline_histogram(
    msname=msname, 
    myplot='data_spectrum', 
    chan_list=[60, 82]
)

### polynomial n=1 continuum

In [ ]:
msname='uvcont_noise_sub_1.ms'

plot_baseline_histogram(
    msname=msname, 
    myplot='data_spectrum', 
    chan_list=[60, 82]
)

### polynomial n=2 continuum

In [ ]:
msname='uvcont_noise_sub_2.ms'

plot_baseline_histogram(
    msname=msname, 
    myplot='data_spectrum', 
    chan_list=[60, 82]
)

### polynomial n=3 continuum

In [ ]:
msname='uvcont_noise_sub_3.ms'

plot_baseline_histogram(
    msname=msname, 
    myplot='data_spectrum', 
    chan_list=[60, 82]
)